In [1]:
import tensorflow as tf
print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.9.1


## Load a dataset

Load and prepare the MNIST dataset. Convert the sample data from integers to floating-point numbers

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 4s 0us/step


## Build a machine learning model

Build a `tf.keras.Sequential` model by stacking layers.

In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

For each example, the model returns a vector of logits or log-odds scores, one for each class.

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.1412586 ,  0.17658943,  0.09407218,  0.6268555 , -0.5381893 ,
         0.12391141,  0.4281706 , -0.30423665, -0.01014163, -0.11453978]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to probabilities for each class:

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.07964782, 0.10944943, 0.10078056, 0.17169656, 0.05355365,
        0.10383308, 0.14075823, 0.06766957, 0.09080657, 0.0818046 ]],
      dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

Define a loss function for training using `losses.SparseCategoricalCrossentropy`, which takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10 ~= 2.3`.

In [7]:
loss_fn(y_train[:1], predictions).numpy()

2.2649708

Before you start training, configure and compile the model using Keras `Model.compile`. Set the `optimizer` class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [9]:
model.compile(optimizer='adam',
             loss = loss_fn,
             metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss:

In [10]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2943 - accuracy: 0.9139
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1433 - accuracy: 0.9577
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1083 - accuracy: 0.9676
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0883 - accuracy: 0.9736
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0741 - accuracy: 0.9760


The `Model.evaluate` method checks the models performance, usually on a "validation-set" or "test-set".

In [11]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - loss: 0.0797 - accuracy: 0.9774 - 425ms/epoch - 1ms/step


[0.07972682267427444, 0.977400004863739]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the TensorFlow tutorials.

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [12]:
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

In [13]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.06895948e-09, 1.91003680e-09, 5.10403197e-07, 3.93191476e-05,
        1.46616243e-11, 1.08053788e-08, 6.31383834e-13, 9.99959826e-01,
        2.10462616e-07, 1.75307264e-07],
       [3.70804165e-09, 1.08913006e-03, 9.98871267e-01, 3.71457609e-05,
        7.44067537e-15, 1.81073597e-06, 1.90129313e-07, 9.64330091e-12,
        3.15331050e-07, 1.28487227e-12],
       [2.34650088e-07, 9.99226570e-01, 4.88180995e-05, 3.61709499e-05,
        6.53945826e-05, 2.24615292e-06, 4.23691245e-06, 4.87104175e-04,
        1.28765838e-04, 5.27540010e-07],
       [9.98920202e-01, 7.05683734e-09, 2.68966629e-04, 1.51864199e-07,
        1.12514647e-06, 3.02767585e-05, 5.16689761e-05, 6.13550365e-04,
        1.87825003e-06, 1.12179296e-04],
       [1.12925989e-06, 1.03796860e-09, 6.88388297e-07, 3.55857876e-09,
        9.95467544e-01, 1.07819176e-06, 1.46742627e-06, 1.69878840e-04,
        2.15641535e-06, 4.35592560e-03]], dtype=float32)>

## Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the Keras API.